# Emergency Networking API - Connection and data fetch

In [4]:
# Import modules
import http.client
import pandas as pd
import json

## Make a connection to the Emergency Networking API and fetch data

This is a simple example of how to connect to the Emergency Networking API and fetch data. The API is a RESTful API that provides access to a database of emergency contacts and resources. The API is free to use and does not require an API key.

In [5]:
# Setup main variables

with open('assets/token.txt', 'r') as file:
    token = file.read()
conn = http.client.HTTPSConnection("staging.emergencynetworking.com")

headers = {
    'Accept': "application/json",
    'Authorization': f"Bearer {token}"
}
print("Requesting data from API")

# API call
conn.request("GET", "/department-api/v/incidents", headers=headers)

res = conn.getresponse()
data = res.read()
data_obj = json.loads(data.decode("utf-8"))

# print dataObject attributes 'total', ''perPage', 'page', 'data'
print("Data received")
print(f"Total records: {data_obj['total']}")
print(f"Records per page: {data_obj['perPage']}")
print(f"Page: {data_obj['currentPage']}")

# transform dataObj['data'] to pandas dataframe
df = pd.DataFrame(data_obj['data'])
df.head(5)


Requesting data from API
Data received
Total records: 22053
Records per page: 25
Page: 1


,id,dispatched_as,station,command_established,location_address_unit,360_evaluation_complete,location_address_unit_type,first_water,primary_search,controlled,...,shift,district,primary_action_taken,additional_actions_taken,narrative,officer_in_charge,units,current_status,created_at,updated_at
0,7c598630-ee34-11ee-9199-ffd8fb9b0f4c,,,1970-01-01T00:00:00.000000Z,None,1970-01-01T00:00:00.000000Z,None,1970-01-01T00:00:00.000000Z,1970-01-01T00:00:00.000000Z,1970-01-01T00:00:00.000000Z,...,,,34,[34],EMS call with pt transport.,1008,[{'id': '9dbe6ac4-18b8-405a-9561-d8cb7ec95e0f'...,archived,2022-09-16T00:48:42.000000Z,2024-07-30T18:25:28.000000Z
1,7c60a400-ee34-11ee-98ae-f561702ccec3,,,1970-01-01T00:00:00.000000Z,None,1970-01-01T00:00:00.000000Z,None,1970-01-01T00:00:00.000000Z,1970-01-01T00:00:00.000000Z,1970-01-01T00:00:00.000000Z,...,,,32,[32],EMS call with pt transport.,1008,[{'id': '386be940-dc67-4727-9a84-89401c026fcb'...,archived,2022-09-16T02:41:13.000000Z,2024-07-30T18:25:28.000000Z
2,7c6826c0-ee34-11ee-8510-f52c0e7fd3fa,,,1970-01-01T00:00:00.000000Z,None,1970-01-01T00:00:00.000000Z,None,1970-01-01T00:00:00.000000Z,1970-01-01T00:00:00.000000Z,1970-01-01T00:00:00.000000Z,...,,,34,[34],Medical transport.,948,[{'id': 'e73bf29c-0d51-4055-b142-27663114742d'...,archived,2022-09-16T02:48:44.000000Z,2024-07-30T18:25:28.000000Z
3,7c6d52e0-ee34-11ee-be6a-e92e937dd693,,,1970-01-01T00:00:00.000000Z,None,1970-01-01T00:00:00.000000Z,None,1970-01-01T00:00:00.000000Z,1970-01-01T00:00:00.000000Z,1970-01-01T00:00:00.000000Z,...,,,34,[34],EMS call and transport.,958,[{'id': 'd9e606a7-0e05-42e5-b923-d62bc8b73e8b'...,archived,2022-09-16T03:01:30.000000Z,2024-07-30T18:25:28.000000Z
4,7c732bd0-ee34-11ee-8057-d3b462e8d235,,,1970-01-01T00:00:00.000000Z,None,1970-01-01T00:00:00.000000Z,None,1970-01-01T00:00:00.000000Z,1970-01-01T00:00:00.000000Z,1970-01-01T00:00:00.000000Z,...,,,33,[33],3171 responded to 911 call and provided medica...,1008,[{'id': '679869bd-3b12-485e-ad4e-788df23cab8a'...,archived,2022-09-16T03:51:00.000000Z,2024-07-30T18:25:28.000000Z


## Data Ingest

Now, we can make the http request iteratively to get all the data. We know the total number of records and the number of records per page. We can calculate the number of pages and make the request for each page.

In [6]:
# Make request to get all pages
total_pages = (data_obj['total'] + data_obj['perPage'] - 1) // data_obj['perPage']

# create an empty array to store all data objects
data_objects = []

# loop through all pages
print(f"Total pages: {total_pages}")
for page in range(1, total_pages + 1):
    try:
        conn.request("GET", f"/department-api/v/incidents?page={page}", headers=headers)
        res = conn.getresponse()
        data = res.read()
        data_obj = json.loads(data.decode("utf-8"))
        #print(data_obj['data']) # uncomment to see the data objects
        # take data_obj['data'] array and append it's content (expand array first to get objects) to the data_objects array
        data_objects.extend(data_obj['data'])
        print(f"Page {page} done")
    except Exception as e:
        print(f"Error occurred on page {page}: {str(e)}")

# print the length of the data_objects array
print(f"Data objects length: {len(data_objects)}")

# validate if the length of the data_objects array is equal to the total records
if len(data_objects) == data_obj['total']:
    print("Data objects length is equal to the total records")

# transform data_objects to pandas dataframe
fire_billing_df = pd.DataFrame(data_objects)
fire_billing_df.head(10)
fire_billing_df.tail(10)

Total pages: 883
Page 1 done
Error occurred on page 2: 'data'
Page 3 done
Page 4 done
Error occurred on page 5: 'data'
Page 6 done
Page 7 done
Page 8 done
Page 9 done
Page 10 done
Page 11 done
Error occurred on page 12: 'data'
Page 13 done
Error occurred on page 14: 'data'
Page 15 done
Page 16 done
Page 17 done
Error occurred on page 18: 'data'
Error occurred on page 19: 'data'
Error occurred on page 20: 'data'
Error occurred on page 21: 'data'
Error occurred on page 22: 'data'
Page 23 done
Page 24 done
Page 25 done
Error occurred on page 26: 'data'
Page 27 done
Page 28 done
Page 29 done
Page 30 done
Page 31 done
Page 32 done
Error occurred on page 33: 'data'
Page 34 done
Error occurred on page 35: 'data'
Page 36 done
Error occurred on page 37: 'data'
Error occurred on page 38: 'data'
Page 39 done
Page 40 done
Page 41 done
Page 42 done
Page 43 done
Page 44 done
Page 45 done
Page 46 done
Page 47 done
Error occurred on page 48: 'data'
Error occurred on page 49: 'data'
Error occurred on p

,id,dispatched_as,station,command_established,location_address_unit,360_evaluation_complete,location_address_unit_type,first_water,primary_search,controlled,...,shift,district,primary_action_taken,additional_actions_taken,narrative,officer_in_charge,units,current_status,created_at,updated_at
14093,c3a239c0-ee18-11ee-ac96-233d0e477727,,,1970-01-01T00:00:00.000000Z,None,1970-01-01T00:00:00.000000Z,None,1970-01-01T00:00:00.000000Z,1970-01-01T00:00:00.000000Z,1970-01-01T00:00:00.000000Z,...,,,34,[34],57 YOM weakness,958,[{'id': 'ff336dae-6e65-47b2-a25e-52820bc9a8f5'...,archived,2022-12-31T03:40:53.000000Z,2024-03-29T22:07:42.000000Z
14094,c3a847e0-ee18-11ee-9e13-bff672deb40c,,,1970-01-01T00:00:00.000000Z,None,1970-01-01T00:00:00.000000Z,None,1970-01-01T00:00:00.000000Z,1970-01-01T00:00:00.000000Z,1970-01-01T00:00:00.000000Z,...,,,34,[34],Transport to SLMC.,1010,[{'id': '5bd025e9-ce0e-4420-86b1-784fa54d9eba'...,archived,2022-12-31T03:56:57.000000Z,2024-03-29T22:07:42.000000Z
14095,c3b2e7a0-ee18-11ee-ab1d-b76bf95057dc,,,1970-01-01T00:00:00.000000Z,None,1970-01-01T00:00:00.000000Z,None,1970-01-01T00:00:00.000000Z,1970-01-01T00:00:00.000000Z,1970-01-01T00:00:00.000000Z,...,,,93,[93],cancelled by fire com,1014,[{'id': 'c2971355-0373-488f-83e8-43461a1ce81f'...,archived,2022-12-31T05:04:55.000000Z,2024-03-29T22:07:42.000000Z
14096,c3ba32f0-ee18-11ee-8f5c-ddafa3e0462a,,,1970-01-01T00:00:00.000000Z,None,1970-01-01T00:00:00.000000Z,None,1970-01-01T00:00:00.000000Z,1970-01-01T00:00:00.000000Z,1970-01-01T00:00:00.000000Z,...,,,31,[31],Pt. refusal.,1011,[{'id': '25129689-80ea-4d49-9507-5d412fb7aba0'...,archived,2022-12-31T06:07:56.000000Z,2024-03-29T22:07:42.000000Z
14097,c3c05fd0-ee18-11ee-9ae6-2bd772a43ce8,,,1970-01-01T00:00:00.000000Z,None,1970-01-01T00:00:00.000000Z,None,1970-01-01T00:00:00.000000Z,1970-01-01T00:00:00.000000Z,1970-01-01T00:00:00.000000Z,...,,,71,[71],"3114 lifted subject from floor to bed, no medi...",1014,[{'id': '96798046-8d80-448e-b8c1-d071af5830eb'...,archived,2022-12-31T06:59:30.000000Z,2024-03-29T22:07:42.000000Z
14098,c3c5e750-ee18-11ee-812e-eb8232f88748,,,1970-01-01T00:00:00.000000Z,None,1970-01-01T00:00:00.000000Z,None,1970-01-01T00:00:00.000000Z,1970-01-01T00:00:00.000000Z,1970-01-01T00:00:00.000000Z,...,,,34,[34],EMS call,1011,[{'id': 'ae7c666d-d510-48a4-8cd2-716bcd392d9e'...,archived,2022-12-31T08:41:52.000000Z,2024-03-29T22:07:42.000000Z
14099,c3cc7500-ee18-11ee-9e1b-db63f615b24c,,,1970-01-01T00:00:00.000000Z,None,1970-01-01T00:00:00.000000Z,None,1970-01-01T00:00:00.000000Z,1970-01-01T00:00:00.000000Z,1970-01-01T00:00:00.000000Z,...,,,93,[93],3173 and 3111 canceled en route. 3173 and 3111...,949,[{'id': 'f8760d4b-cf6c-48c8-ac21-4039fd0c6444'...,archived,2022-12-31T08:42:00.000000Z,2024-03-29T22:07:42.000000Z
14100,c2c10150-ee18-11ee-ab89-c71bbd81f514,,,1970-01-01T00:00:00.000000Z,None,1970-01-01T00:00:00.000000Z,None,1970-01-01T00:00:00.000000Z,1970-01-01T00:00:00.000000Z,1970-01-01T00:00:00.000000Z,...,,,93,[93],Dispatched for a possible full code in the par...,947,[{'id': 'd7e76871-74e3-4562-905a-c503b1eda48b'...,archived,2022-12-23T02:49:28.000000Z,2024-03-29T22:07:40.000000Z
14101,c2c63ca0-ee18-11ee-97b9-491245277e2d,,,1970-01-01T00:00:00.000000Z,None,1970-01-01T00:00:00.000000Z,None,1970-01-01T00:00:00.000000Z,1970-01-01T00:00:00.000000Z,1970-01-01T00:00:00.000000Z,...,,,71,[71],Assist female that was 1 day post knee replace...,947,[{'id': '837b27c0-b40f-4596-9991-463d470ef153'...,archived,2022-12-23T03:37:28.000000Z,2024-03-29T22:07:40.000000Z
14102,c2cbae20-ee18-11ee-bdd8-9d7b9ea9b4f0,,,1970-01-01T00:00:00.000000Z,None,1970-01-01T00:00:00.000000Z,None,1970-01-01T00:00:00.000000Z,1970-01-01T00:00:00.000000Z,1970-01-01T00:00:00.000000Z,...,,,33,[33],"Medical call, with no transport. See PCR.",1010,[{'id': 'bfeb5b1e-e6c2-47fd-a2d7-a189cff7c3d5'...,archived,2022-12-23T10:50:54.000000Z,2024-03-29T22:07:40.000000Z


## Data stats and charts

Here we will show some basic statistics and charts based on the data we have fetched.

In [7]:
# Basic stats and charts

# describe the data
#fire_billing_df.describe()

# aggregate the data by 'incident_type' and count the number of records
#fire_billing_df.groupby(['Incident Type', 'Address', 'Incident Narrative']).agg({'id': 'count'}).rename(columns={'id': 'Total Incidents'}).sort_values('Total Incidents', ascending=False).reset_index()


KeyError: 'Incident Type'

## Data modeling and exporting

In [5]:
# Table transformations and key propagation

In [8]:
#export the fire_billing_df to .csv file
fire_billing_df.to_csv('assets/incidents.csv', index=False)

#export the fire_billing_df to .json file
fire_billing_df.to_json('assets/incidents.json', orient='records')